## Preprocessing

Adding more neurons and deleted columns

In [24]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN","NAME"], axis=1, inplace=True)

In [26]:
application_df["SPECIAL_CONSIDERATIONS"].value_counts()


N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [27]:
application_df["STATUS"].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [28]:
application_df.drop(["SPECIAL_CONSIDERATIONS","STATUS"], axis=1, inplace=True)

In [29]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [30]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [31]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

cut_off_value = 500    #replace types with count less than 500
application_types_to_replace = []

#get list of all application values
cut_vals = list(application_df['APPLICATION_TYPE'].unique())
for cut in cut_vals:
  #get application values below the cut off
  if (application_df['APPLICATION_TYPE'].value_counts()[cut] < cut_off_value):
    application_types_to_replace.append(cut)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [32]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [33]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts().loc[lambda x : x > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [34]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1850    #replace types with count less than 1850
classifications_to_replace = []

#get list of all classification values
cut_vals = list(application_df['CLASSIFICATION'].unique())
for cut in cut_vals:
  #get classification values below the cut off
  if (application_df['CLASSIFICATION'].value_counts()[cut] < cutoff_value):
    classifications_to_replace.append(cut)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [35]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_df = pd.get_dummies(application_df)
categorical_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [36]:
# Split our preprocessed data into our features and target arrays
# Separate the y variable, the labels
y = categorical_df["IS_SUCCESSFUL"].values.reshape(-1, 1)

# Separate the X variable, the features
X = categorical_df.copy()
X.drop("IS_SUCCESSFUL", axis=1, inplace=True)


In [37]:
# Review the X variable DataFrame
X.head()

,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [38]:
# Review the y variable Series
y[:5]

array([[1],
       [1],
       [0],
       [1],
       [1]])

In [39]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)


In [40]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=40))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3280      
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 6,561
Trainable params: 6,561
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
#Add callback to save for every 5 epochs

checkpoint = ModelCheckpoint("model_weights5.h5", monitor='accuracy', verbose=1,
    save_weights_only=True, mode='auto', period=5)


In [44]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100,callbacks=[checkpoint])


Epoch 1/100
751/751 [==============================] - 3s 2ms/step - loss: 0.5704 - accuracy: 0.7197
Epoch 2/100
751/751 [==============================] - 3s 3ms/step - loss: 0.5564 - accuracy: 0.7276
Epoch 3/100
751/751 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7300
Epoch 4/100
751/751 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7294
Epoch 5/100
740/751 [============================>.] - ETA: 0s - loss: 0.5499 - accuracy: 0.7314
Epoch 5: saving model to model_weights5.h5
751/751 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7318
Epoch 6/100
751/751 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7309
Epoch 7/100
751/751 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7307
Epoch 8/100
751/751 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7315
Epoch 9/100
751/751 [==============================] - 2s 3ms/st

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 1s - loss: 0.5572 - accuracy: 0.7320 - 578ms/epoch - 2ms/step
Loss: 0.5572029948234558, Accuracy: 0.7319728136062622


In [46]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity5.h5")